In [ ]:
im

In [ ]:
##########################################################
# Step 3: Save Detecter

os.chdir("/Users/Bya/Dropbox/Research/datas/Detecter/")
with open('log_tweets_pn_detector-20160114.pkl', 'wb') as fout:
    pickle.dump(log_detector, fout)